In [17]:
import nltk
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer
from pandas import Series, DataFrame
import pandas as pd
%config InlineBackend.figure_format = 'retina'
%pylab inline
from IPython.core.display import display, HTML
display(HTML("<style>.container {width:100% !important;}</style>"))
from nltk.corpus import stopwords
from __future__ import division  # Python 2 users only
import nltk, re, pprint
from nltk import word_tokenize
import random
from nltk.util import ngrams
from collections import Counter
from nltk.probability import FreqDist
from sklearn import metrics

from nltk.classify import SklearnClassifier
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

**Deliverables
The deliverable for this assignment is a file with the python scripts, outputs (including plots & tables where applicable) and your answers to the various questions. Do not forget to write the names of all group members. **

In [33]:
df_data = pd.read_csv('Train_rev1.csv')

In [34]:
df_data[:3]

,Id,Title,FullDescription,LocationRaw,LocationNormalized,ContractType,ContractTime,Company,Category,SalaryRaw,SalaryNormalized,SourceName
0,12612628,Engineering Systems Analyst,Engineering Systems Analyst Dorking Surrey Sal...,"Dorking, Surrey, Surrey",Dorking,NaN,permanent,Gregory Martin International,Engineering Jobs,20000 - 30000/annum 20-30K,25000,cv-library.co.uk
1,12612830,Stress Engineer Glasgow,Stress Engineer Glasgow Salary **** to **** We...,"Glasgow, Scotland, Scotland",Glasgow,NaN,permanent,Gregory Martin International,Engineering Jobs,25000 - 35000/annum 25-35K,30000,cv-library.co.uk
2,12612844,Modelling and simulation analyst,Mathematical Modeller / Simulation Analyst / O...,"Hampshire, South East, South East",Hampshire,NaN,permanent,Gregory Martin International,Engineering Jobs,20000 - 40000/annum 20-40K,30000,cv-library.co.uk


In [35]:
import sys  

reload(sys)  
sys.setdefaultencoding('utf8')
print 'hi'

In [36]:
top_percentile = df_data.SalaryNormalized>df_data.SalaryNormalized.quantile(.75)
df_data['BinarySalary'] = np.where(top_percentile, 1, 0)
b_salary = df_data['BinarySalary']
df_data['BinarySalary'].value_counts()
b_salary

0         0
1         0
2         0
3         0
4         0
5         0
6         1
7         0
8         0
9         1
10        0
11        0
12        1
13        0
14        0
15        0
16        1
17        0
18        0
19        0
20        0
21        0
22        0
23        0
24        0
25        0
26        0
27        0
28        0
29        0
         ..
244738    0
244739    1
244740    1
244741    0
244742    0
244743    0
244744    0
244745    1
244746    1
244747    0
244748    0
244749    0
244750    0
244751    0
244752    0
244753    0
244754    0
244755    0
244756    0
244757    0
244758    0
244759    0
244760    0
244761    0
244762    0
244763    0
244764    0
244765    0
244766    0
244767    0
Name: BinarySalary, Length: 244768, dtype: int64

In [23]:
####RCD -we probably need to take a random sample of the DF, so we can get the right salaries to match up for Part B 

smaller_df = df_data.sample(1000, random_state=100)
del df_data

In [24]:
job_desc_list=smaller_df.FullDescription.values.tolist()
job_desc_list[0] #the first job description (complete)
words=[]
for i in range(0,len(job_desc_list)):
    words1=word_tokenize(job_desc_list[i])
    words.append(words1)

In [25]:
words[0]

['A',
 'Marketing',
 'Executive',
 'is',
 'required',
 'to',
 'join',
 'a',
 'global',
 'value',
 'added',
 'distributor',
 'of',
 'unified',
 'communications',
 'currently',
 'based',
 'in',
 'Burgess',
 'Hill',
 '.',
 'You',
 'will',
 'have',
 'the',
 'responsibility',
 'for',
 'day',
 'to',
 'day',
 'marketing',
 'within',
 'the',
 'department',
 'as',
 'defined',
 'by',
 'the',
 'activities',
 'below',
 '.',
 'This',
 'role',
 'is',
 'as',
 'an',
 'integral',
 'member',
 'of',
 'a',
 'small',
 'but',
 'busy',
 'marketing',
 'team',
 '.',
 'Key',
 'attributes',
 'will',
 'to',
 'be',
 'proactively',
 'engaged',
 'with',
 'the',
 'GTM',
 'marketing',
 'activities',
 'and',
 'business',
 'touchpoints',
 'that',
 'marketing',
 'works',
 'internally',
 'with',
 'vendors',
 'and',
 'with',
 'customers',
 'when',
 'required',
 '.',
 'The',
 'ideal',
 'candidate',
 ',',
 'will',
 'enjoy',
 'working',
 'across',
 'a',
 'broad',
 'marketing',
 'mix',
 '.',
 'They',
 'will',
 'be',
 'involved

In [ ]:
job_words = [item for sublist in words for item in sublist]
job_words

In [ ]:
len(job_words)

In [ ]:
job_desc.shape

# Part A

**A1. What are the top 5 parts of speech in this corpus of job descriptions? How frequently do they appear?**

Hint: nltk.org is a great resource for exploring text mining with Python. There are many examples that are similar to the questions in this assignment.  

In [ ]:
tagged = nltk.pos_tag(job_words)
counts = Counter(tag for word,tag in tagged)
counts.most_common()[:5]

#the most common parts of speech are 
#NN: noun, common/singular/mass

**A2. Does this corpus support Zipf’s law? Plot the most common 100 words in the corpus against the theoretical prediction of the law. For this question, do not remove stopwords. Also do not perform stemming or lemmatization.**

Hint: Check http://www.garysieling.com/blog/exploring-zipfs-law-with-python-nltk-scipy-and-matplotlib 

In [ ]:
freqdist = FreqDist(job_words)
top_100=freqdist.most_common(100)

freqdist.plot(50, cumulative=False)

print top_100

**A3. If we remove stopwords and lemmatize the corpus, what are the 10 most common words? What are their frequencies?**


In [ ]:
filtered_words = [word for word in job_words if word not in stopwords.words('english')]  ##I think filtered words wasn't working before because we hadn't tokenized
filtered_words = [w.lower() for w in filtered_words if w.isalpha()]
#filtered_words
filtered_freq = FreqDist(filtered_words)
filtered_freq.most_common(10)
##'The' will be removed once we get our 'words' list into lowercase

# Part B (predict salary from job description)

In this section, you will create classification models to predict high (75th percentile and above) or low (below 75th percentile) salary from the text contained in the job descriptions. Ignore all other data provided on kaggle.com that is not text (except the actual salary data, which you need to create the binary output of high/low salary).    
Get the training data on kaggle.com (train_rev1) (do not use the validation data provided on kaggle.com). Use the Naïve Bayes classifier to classify job descriptions into high and low salary categories (we will discuss the theory of Naïve Bayes later in this course). There are two versions of Naïve Bayes – binomial and multinomial; for the moment, let’s not bother about the theoretical underpinnings! You can use either one for this assignment. For all models below, show the confusion matrix.
Hint: For part B, check out   http://www.nltk.org/book/ch06.html (esp 1.3) for illustrations.

Also look at http://nbviewer.ipython.org/gist/rjweiss/7158866 (you may have to download additional Python libraries than the ones I have mentioned before) 

**B1. Create a classification model with all words and the bag-of-words approach. How accurate is the model (show the confusion matrix)? **

In [28]:
trainset_size = int(round(len(job_desc_list)*0.75))
X_train = np.array([''.join(el) for el in job_desc_list[:trainset_size]])
y_train = np.array([el for el in smaller_df.BinarySalary[:trainset_size]])
X_test = np.array([''.join(el) for el in job_desc_list[trainset_size:]])
y_test = np.array([el for el in smaller_df.BinarySalary[trainset_size:]])


In [29]:
pipeline = Pipeline([('tfidf', TfidfTransformer()),
                     ('chi2', SelectKBest(chi2, k=1000)),
                     ('nb', MultinomialNB())])
classif = SklearnClassifier(pipeline)


In [ ]:
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)
nb_classifier = MultinomialNB().fit(X_train_vec, y_train)

y_nb_predicted = nb_classifier.predict(X_test_vec)


In [ ]:
str(metrics.precision_score(y_test, y_nb_predicted))
str(metrics.recall_score(y_test, y_nb_predicted))

In [ ]:
X_train_vec

In [31]:
cl = nltk.NaiveBayesClassifier.train(train_set)
print "%.3f" % nltk.classify.accuracy(cl, test_set)
cl.show_most_informative_features(40)
cl.prob_classify(featurize(name)) # get a confidence for the prediction

TypeError: 'SklearnClassifier' object is not callable

In [ ]:
metrics.confusion_matrix(y_test, y_nb_predicted)


In [ ]:
nltk.ConfusionMatrix(gold, test)

**B2. Speculate before running the following analysis whether lemmatization would help improve the accuracy of classification. Now create a classification model after lemmatization. Did the classification accuracy increase relative to B1? Comment on your speculation versus the actual results you obtained.**


**B3. Now speculate whether stopwords removal from the original data would help increase the accuracy of the model. Take out the stopwords (but do not lemmatize), build a classification model and check the accuracy, and compare with that in B1 & B2.   
Also show the top 10 words (excluding stopwords) that are most indicative of (i) high salary, and (ii) low salary.** 

**B4.  Use the job descriptions without lemmatiztion and stopword removal. Add parts-of-speech bigrams to the bag-of-words, and run a new classification model. Does the accuracy increase over the results in B1?  **